In [2]:
#ライブラリ読み込み
from IPython.display import clear_output
from selenium import webdriver
import time
from pprint import pprint
from Bio import AlignIO
#相対パスを指定

file_path = 'Sympetrum_consensus/Sympetrum_kunckeli_consensus.fasta'
target = AlignIO.read(file_path, 'fasta')
print(target.format('fasta'))


#クロームの立ち上げ
driver=webdriver.Chrome()

#URLの指定
driver.get("https://www.bioinformatics.nl/cgi-bin/primer3plus/primer3plus.cgi")
time.sleep(1)

#検索ボックスの指定
search_box=driver.find_element_by_xpath("/html/body/div/div[2]/form/div[5]/div[3]/table[1]/tbody/tr[2]/td/textarea")

#検索ボックスに文字を入力
search_box.send_keys(target.format('fasta'))

#検索ボタンの指定
search_button=driver.find_element_by_xpath("/html/body/div/div[2]/form/div[2]/table/tbody/tr/td[3]/table/tbody/tr/td[1]/input")

#検索ボタンをクリック
search_button.click()
time.sleep(0.5)

#ブラウザのウインドウ高を取得する
win_height = driver.execute_script("return window.innerHeight")

#スクロール開始位置の初期値（ページの先頭からスクロールを開始する）
last_top = 1

#ページの最下部までスクロールする無限ループ
while True:

  #スクロール前のページの高さを取得
  last_height = driver.execute_script("return document.body.scrollHeight")
  
  #スクロール開始位置を設定
  top = last_top

  #ページ最下部まで、徐々にスクロールしていく
  while top < last_height:
    top += int(win_height * 0.1)
    driver.execute_script("window.scrollTo(0, %d)" % top)
    time.sleep(0.05)

  #１秒待って、スクロール後のページの高さを取得する
  time.sleep(1)
  new_last_height = driver.execute_script("return document.body.scrollHeight")

  #スクロール前後でページの高さに変化がなくなったら無限スクロール終了とみなしてループを抜ける
  if last_height == new_last_height:
    break

  #次のループのスクロール開始位置を設定
  last_top = last_height

# primer dict
primers = {}
i = 0
x = 0
while True:
    try:
        forward_primer = driver.find_element_by_id(f"PRIMER_{i}_SEQUENCE").get_attribute("value")
        i += 1
        x += 1
        reverse_primer = driver.find_element_by_id(f"PRIMER_{i}_SEQUENCE").get_attribute("value")
        primers[f"primer{x}"] = {"forward primer": forward_primer, "reverse primer": reverse_primer}
        i += 1
    except:
        break

clear_output(wait=True)
pprint(primers)

#クロームの終了
driver.quit()


{'primer1': {'forward primer': 'TATAGCATTCCCCCGATTGA',
             'reverse primer': 'CACTCCCGCAAGATGTAATG'},
 'primer2': {'forward primer': 'CATTACATCTTGCGGGAGTG',
             'reverse primer': 'ATAGCACCAGCCAGAACAGG'},
 'primer3': {'forward primer': 'TATAGCATTCCCCCGATTGA',
             'reverse primer': 'TCCCGCAAGATGTAATGAAA'},
 'primer4': {'forward primer': 'CATTACATCTTGCGGGAGTG',
             'reverse primer': 'GAATTGGGTCTCCACCTCCT'},
 'primer5': {'forward primer': 'TATAGCATTCCCCCGATTGA',
             'reverse primer': 'ACACTCCCGCAAGATGTAATG'}}


Needleman-Wunschアルゴリズムについて
https://bi.biopapyrus.jp/seq/alignment/needleman%E2%80%93wunsch.html

In [4]:
from Bio import pairwise2
from Bio.Seq import Seq
#候補に上がったprimerと、target種の配列データを比較する
#file_path = '種特異にしたい種の配列データ'のパス
file_path = '/home/dkcom/デスクトップ/primer_design/alignment_python/Sympetrum/Sympetrum_kunckeli.fasta'
target = AlignIO.read(file_path, 'fasta')

# 配列データの比較
for primer_num, primer in primers.items():
    for record in target:
        for i in range(len(record.seq) - len(primer["forward primer"])):
            alignments = pairwise2.align.globalxx(primer["forward primer"], str(record.seq[i:i+len(primer["forward primer"])]))
            if alignments[0][2] / alignments[0][4] >= 1.0:
                print(f"{primer_num}'s forward primer is found in related species sequences.")
                break
        else:
            print("\033[31m{}\033[0m".format(f"{primer_num}'s forward primer is not found in related species sequences."))
    # reverse primerを相補鎖に変換
    rev_primer = Seq(primer["reverse primer"]).reverse_complement()
    for record in target: 
        for i in range(len(record.seq) - len(rev_primer)):
            alignments = pairwise2.align.globalxx(rev_primer, str(record.seq[i:i+len(rev_primer)]))
            if alignments[0][2] / alignments[0][4] >= 1.0:
                print(f"{primer_num}'s reverse primer is found in related species sequences.")
                break
        else:
            print("\033[31m{}\033[0m".format(f"{primer_num}'s reverse primer is not found in related species sequences."))


primer1's forward primer is not found in related species sequences.
primer1's forward primer is found in related species sequences.
primer1's forward primer is not found in related species sequences.
primer1's forward primer is not found in related species sequences.
primer1's reverse primer is found in related species sequences.
primer1's reverse primer is found in related species sequences.
primer1's reverse primer is found in related species sequences.
primer1's reverse primer is not found in related species sequences.
primer2's forward primer is found in related species sequences.
primer2's forward primer is found in related species sequences.
primer2's forward primer is found in related species sequences.
primer2's forward primer is not found in related species sequences.
primer2's reverse primer is found in related species sequences.
primer2's reverse primer is found in related species sequences.
primer2's reverse primer is found in related species sequences.
primer2's reverse pr